# **Economic Development vs. Sustainability**
# Joining Datasets - CO_2 Emissions and GDP
Katlyn Goeujon-Mackness <br>
Last Updated: 20/06/2025

## Table of Contents

**Introduction**  
- Purpose of the joined dataset  
- Overview of source datasets (GDP, emissions, population, metadata)

**Loading Cleaned Inputs**  
- Load emissions data  
- Load GDP data  
- Load population and regional metadata

**Initial Validations**  
- Shape and key stats comparison  
- ISO code and year coverage  
- Data completeness checks

**Sequential Merging Process**  
- Merge GDP with emissions  
- Add population data  
- Join region/economic group info  
- Post-merge validation at each step

**Data Harmonization**  
- Filtering aggregates and invalid rows  
- Ensuring consistent data types  
- Managing duplicates and nulls

**Feature Engineering**  
- Create log-transformed features  
- Calculate per-capita and intensity metrics  
- Categorize emissions or GDP tiers

**Final Validation**  
- Summary stats for final dataset  
- Spot check samples  
- Row evolution across steps

**Output and Export**  
- Save structured dataset to CSV  
- Notes for downstream use


## Introduction

Economic growth is often pursued at the cost of environmental sustainability. This study aims to analyze the balance between economic development and sustainable practices across different regions, industries, and policies.

In this **data cleaning phase**, we will refine and structure the dataset by handling missing values, removing redundant entries, and ensuring consistency in country classifications and ISO codes. This process will improve data quality, enabling accurate analysis. Once cleaned, we will export the processed dataset in CSV format for further exploration.

### Key Challenge
Achieving sustainable economic growth requires balancing financial prosperity with environmental and social responsibility. Identifying actionable patterns in historical data can inform policymakers, businesses, and environmental advocates.

### Data of Interest
- GDP growth rate compared to carbon emissions per capita (current analysis).
- Percentage of renewable energy adoption.
- Employment trends in green industries.
- Improvement in environmental quality indicators (air quality, water safety).
- Sustainability index scores vs. economic performance.

### Locating Relevant Data
- **World Bank**: Economic indicators.
    * [GDP per capita growth (annual %)](https://data.worldbank.org/indicator/NY.GDP.PCAP.KD.ZG)
    * [GDP per capita (constant 2015 US$)](https://data.worldbank.org/indicator/NY.GDP.PCAP.KD)
- **Our World in Data**: Environmental indicators and population data
    * [CO2 and Greenhouse Gas Emissions](https://github.com/owid/co2-data)
    * [Historical and Projected Population](https://ourworldindata.org/population-sources) 
- **United Nations SDGs Database**: Sustainable development statistics.
- **OECD**: Policy effectiveness on sustainability.
- **NASA Earth Observations**: Environmental impact metrics.
- **National Employment Data**: Job growth in sustainable sectors.

In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import zscore

# Prevent truncating columns and rows
pd.set_option("display.max_rows", None) 
pd.set_option("display.max_columns", None) 

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [58]:
# Load GDP Data
gdp_by_groups = pd.read_csv("../data/in_process/2a_gdp_by_groups_cleaned.csv")
gdp_by_country = pd.read_csv("../data/in_process/2a_gdp_by_country_cleaned.csv")

# Load Emissions Data
co2_by_groups = pd.read_csv("../data/in_process/2b_co2_by_groups_cleaned.csv")
co2_by_country = pd.read_csv("../data/in_process/2b_co2_by_country_cleaned.csv")

In [59]:
# Drop extraneous index columns
gdp_by_groups.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
gdp_by_groups.head(3)

,Country Code,Region Group Name,Year,GDPAnnualPercent,GDPAnnualPercent_winsorized,GDPConstantUSD,GDPConstantUSD_log
0,AFE,Africa Eastern and Southern,1960,-2.136630,-2.136630,1172.316285,7.067589
1,AFE,Africa Eastern and Southern,1961,-2.136630,-2.136630,1147.268217,7.046010
2,AFE,Africa Eastern and Southern,1962,5.009835,5.009835,1204.744460,7.094852


In [60]:
# Drop extraneous index columns
gdp_by_country.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
gdp_by_country.head(3)

,Country Code,Country Name,Year,GDPConstantUSD,GDPConstantUSD_log,GDPAnnualPercent,GDPAnnualPercent_winsorized
0,AFG,Afghanistan,1961,308.31827,5.734371,-10.119484,-10.119484
1,AFG,Afghanistan,1962,308.31827,5.734371,-10.119484,-10.119484
2,AFG,Afghanistan,1963,308.31827,5.734371,-10.119484,-10.119484


In [61]:
# Drop extraneous index columns
co2_by_groups.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
co2_by_groups.head(3)

,Region/Group,Country Code,Year,Continent,Population,CO2,Per Capita CO2,Cumulative CO2,Emissions Category,CO2 Log,Pop Log,Pop Outliers,Pop Category
0,Africa,AFR,1960,Africa,283922289.0,156.567,5.514431e-07,49714.657,Low,5.059851,19.464211,not outlier,100M-1B
1,Africa,AFR,1961,Africa,290814083.0,161.994,5.570363e-07,49714.657,Low,5.093713,19.488195,not outlier,100M-1B
2,Africa,AFR,1962,Africa,297959967.0,166.335,5.582461e-07,49714.657,Low,5.119998,19.512470,not outlier,100M-1B


In [62]:
# Drop extraneous index columns
co2_by_country.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
co2_by_country.head(3)

,Country Name,Country Code,Year,Population,CO2,CO2 Outliers,Per Capita CO2,Cumulative CO2,Emissions Category,CO2 Log,Pop Log,Pop Outliers,Pop Category
0,Afghanistan,AFG,1960,9035048.0,0.414,False,4.582156e-08,235.001,Moderate,0.346423,16.016622,not outlier,1M-10M
1,Afghanistan,AFG,1961,9214082.0,0.491,False,5.328800e-08,235.001,Moderate,0.399447,16.036244,not outlier,1M-10M
2,Afghanistan,AFG,1962,9404411.0,0.689,False,7.326349e-08,235.001,Moderate,0.524137,16.056689,not outlier,1M-10M


---

## Join Datasets by Country

In [63]:
gdp_co2_country = co2_by_country.merge(
    gdp_by_country,
    on=['Country Code', 'Country Name', 'Year'],
    how='inner',
    indicator=True
)
gdp_co2_country["_merge"].value_counts()

_merge
both          12507
left_only         0
right_only        0
Name: count, dtype: int64

In [64]:
gdp_co2_country.head(3)

,Country Name,Country Code,Year,Population,CO2,CO2 Outliers,Per Capita CO2,Cumulative CO2,Emissions Category,CO2 Log,Pop Log,Pop Outliers,Pop Category,GDPConstantUSD,GDPConstantUSD_log,GDPAnnualPercent,GDPAnnualPercent_winsorized,_merge
0,Afghanistan,AFG,1961,9214082.0,0.491,False,5.328800e-08,235.001,Moderate,0.399447,16.036244,not outlier,1M-10M,308.31827,5.734371,-10.119484,-10.119484,both
1,Afghanistan,AFG,1962,9404411.0,0.689,False,7.326349e-08,235.001,Moderate,0.524137,16.056689,not outlier,1M-10M,308.31827,5.734371,-10.119484,-10.119484,both
2,Afghanistan,AFG,1963,9604491.0,0.707,False,7.361140e-08,235.001,Moderate,0.534737,16.077741,not outlier,1M-10M,308.31827,5.734371,-10.119484,-10.119484,both


### Final Clean-Up
One more check and tidy-up for the output dataset.

In [65]:
# Drop merge column
gdp_co2_country = gdp_co2_country.drop(columns=["_merge"])

# Look at DataFrame overview for nulls and datatypes
gdp_co2_country.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12507 entries, 0 to 12506
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Country Name                 12507 non-null  object 
 1   Country Code                 12507 non-null  object 
 2   Year                         12507 non-null  int64  
 3   Population                   12507 non-null  float64
 4   CO2                          12507 non-null  float64
 5   CO2 Outliers                 12507 non-null  object 
 6   Per Capita CO2               12507 non-null  float64
 7   Cumulative CO2               12507 non-null  float64
 8   Emissions Category           12507 non-null  object 
 9   CO2 Log                      12507 non-null  float64
 10  Pop Log                      12507 non-null  float64
 11  Pop Outliers                 12507 non-null  object 
 12  Pop Category                 12507 non-null  object 
 13  GDPConstantUSD  

In [66]:
# Shorten long column names
gdp_co2_country.rename(columns={
    "GDPConstantUSD": "GDP USD",
    "GDPConstantUSD_log": "GDP USD Log",
    "GDPAnnualPercent": "GDP %",
    "GDPAnnualPercent_winsorized": "GDP % Winsor"
    }, inplace=True)

# Preview changes
gdp_co2_country.tail(3)

,Country Name,Country Code,Year,Population,CO2,CO2 Outliers,Per Capita CO2,Cumulative CO2,Emissions Category,CO2 Log,Pop Log,Pop Outliers,Pop Category,GDP USD,GDP USD Log,GDP %,GDP % Winsor
12504,Zimbabwe,ZWE,2021,15797220.0,10.204,False,6.459364e-07,672.114,High,2.416271,16.575345,not outlier,10M-100M,1305.220113,7.174893,6.611911,6.611911
12505,Zimbabwe,ZWE,2022,16069061.0,10.425,False,6.487622e-07,672.114,High,2.435804,16.592406,not outlier,10M-100M,1361.914530,7.217381,4.343667,4.343667
12506,Zimbabwe,ZWE,2023,16340829.0,11.164,False,6.831967e-07,672.114,High,2.498481,16.609177,not outlier,10M-100M,1410.737311,7.252576,3.584864,3.584864


In [67]:
# Add Derived Columns
# CO2 per GDP - emissions efficiency metric (lower is more efficient)
gdp_co2_country['CO2 Per GDP'] = gdp_co2_country['CO2'] / gdp_co2_country['GDP USD']

# GDP per capita
gdp_co2_country['GDP Per Capita'] = gdp_co2_country['GDP USD'] / gdp_co2_country['Population']

# CO2 year on year growth %
gdp_co2_country['CO2 %'] = (
    gdp_co2_country
    .sort_values(['Country Code', 'Year'])
    .groupby('Country Code')['CO2']
    .pct_change() * 100
)

# GDP categories (per World Bank)
bins = [0, 1e9, 1e11, 1e13]
labels = ['Low GDP', 'Middle GDP', 'High GDP']

gdp_co2_country["GDP Category"] = pd.cut(
    gdp_co2_country["GDP USD"],
    bins=bins,
    labels=labels,
    include_lowest=True
)

# Preview sample of the data
gdp_co2_country.sample(5, random_state=1)

,Country Name,Country Code,Year,Population,CO2,CO2 Outliers,Per Capita CO2,Cumulative CO2,Emissions Category,CO2 Log,Pop Log,Pop Outliers,Pop Category,GDP USD,GDP USD Log,GDP %,GDP % Winsor,CO2 Per GDP,GDP Per Capita,CO2 %,GDP Category
9063,Poland,POL,1984,36905796.0,432.467,True,0.000012,21815.830,Very High,6.071816,17.423879,not outlier,10M-100M,5111.345330,8.539414,-7.344791,-7.344791,0.084609,0.000138,2.717897,Low GDP
10198,Slovak Republic,SVK,1987,5205882.0,58.863,False,0.000011,2892.906,High,4.092059,15.465300,not outlier,1M-10M,8597.463999,9.059339,-14.639960,-14.639960,0.006847,0.001651,-1.173567,Low GDP
11981,United States,USA,2001,284279629.0,5907.740,extreme,0.000021,315589.222,Very High,8.684188,19.465469,not outlier,100M-1B,48597.424614,10.791346,-0.038732,-0.038732,0.121565,0.000171,-1.703722,Low GDP
5633,Italy,ITA,1979,56132381.0,385.256,True,0.000007,23367.924,Very High,5.956500,17.843223,not outlier,10M-100M,21219.266957,9.962712,5.653224,5.653224,0.018156,0.000378,3.765952,Low GDP
6344,Lebanon,LBN,1997,4108409.0,15.099,False,0.000004,769.408,High,2.778757,15.228547,not outlier,1M-10M,6008.758554,8.701140,-0.755972,-0.755972,0.002513,0.001463,18.395672,Low GDP


In [68]:
# Reorganize columns in a logical order
gdp_co2_country.columns

Index(['Country Name', 'Country Code', 'Year', 'Population', 'CO2',
       'CO2 Outliers', 'Per Capita CO2', 'Cumulative CO2',
       'Emissions Category', 'CO2 Log', 'Pop Log', 'Pop Outliers',
       'Pop Category', 'GDP USD', 'GDP USD Log', 'GDP %', 'GDP % Winsor',
       'CO2 Per GDP', 'GDP Per Capita', 'CO2 %', 'GDP Category'],
      dtype='object')

In [69]:
cols = ['Country Name', 'Country Code', 'Year', 'Population', 'Pop Log', 'Pop Outliers', 'Pop Category',
        'CO2', 'CO2 %', 'Per Capita CO2', 'Cumulative CO2', 'CO2 Log', 'CO2 Outliers',
        'Emissions Category', 'GDP USD', 'GDP USD Log', 'GDP %', 'GDP % Winsor', 'GDP Per Capita', 'GDP Category',
        'CO2 Per GDP',
        ]
gdp_co2_country = gdp_co2_country[cols]

# Preview changes
gdp_co2_country.head(3)

,Country Name,Country Code,Year,Population,Pop Log,Pop Outliers,Pop Category,CO2,CO2 %,Per Capita CO2,Cumulative CO2,CO2 Log,CO2 Outliers,Emissions Category,GDP USD,GDP USD Log,GDP %,GDP % Winsor,GDP Per Capita,GDP Category,CO2 Per GDP
0,Afghanistan,AFG,1961,9214082.0,16.036244,not outlier,1M-10M,0.491,NaN,5.328800e-08,235.001,0.399447,False,Moderate,308.31827,5.734371,-10.119484,-10.119484,0.000033,Low GDP,0.001593
1,Afghanistan,AFG,1962,9404411.0,16.056689,not outlier,1M-10M,0.689,40.325866,7.326349e-08,235.001,0.524137,False,Moderate,308.31827,5.734371,-10.119484,-10.119484,0.000033,Low GDP,0.002235
2,Afghanistan,AFG,1963,9604491.0,16.077741,not outlier,1M-10M,0.707,2.612482,7.361140e-08,235.001,0.534737,False,Moderate,308.31827,5.734371,-10.119484,-10.119484,0.000032,Low GDP,0.002293


---

## Join Datasets by Region
Due to limitations that arose with the definition of "Region/Group" over the course of the discovery and cleaning processes, a cohesive dataset by region/group based on the current datasets isn't feasible. More specifically, the definition of "Region/Group" varied significantly between the World Bank and Our World in Data datasets, with World Bank offering much more granularity.

For the moment, we will continue the GDP/CO2 analysis without joining these sets, focusing on analysis by country. For future analyses, we can investigate again the regional and economic groupings provided by World Bank.

---

## Export Joined Datasets

In [ ]:
# Comment out to avoid duplicate exports
# gdp_co2_country.to_csv("../data/processed/gdp_co2_by_country_final.csv", index=False)